In [1]:
import cv2
import mediapipe as mp
import math

mp_pose = mp.solutions.pose

# Symmetry (Bad approach)

In [6]:
def is_pose_symmetric(landmarks, threshold=3.65):
    left_landmarks = [landmarks[i] for i in [13,15]]  # Indices for left side
    right_landmarks = [landmarks[i] for i in [14,16]]  # Indices for right side
    symmetry_axis=landmarks[12].x-landmarks[11].x
    
    asymmetry_score=abs(landmarks[0].x-symmetry_axis)
    
    for left, right in zip(left_landmarks, right_landmarks):
        asymmetry_score = asymmetry_score + abs((symmetry_axis-left.x) - (right.x - symmetry_axis))
        asymmetry_score = asymmetry_score + abs(left.y - right.y)

    if(asymmetry_score<threshold):
        return True
    else:
        return False

In [7]:
cap = cv2.VideoCapture('GestureFeatureResources/video.mov')
#cap = cv2.VideoCapture(0)

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      
        # Make detection
        results = pose.process(image)
    
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass
        
        # Indices for the nose, shoulders, elbows, and wrists
        nose_index = 0
        left_shoulder_index = 11
        left_elbow_index = 13
        left_wrist_index = 15
        right_shoulder_index = 12
        right_elbow_index = 14
        right_wrist_index = 16
        
        # Draw circles at specific landmarks
        for index in [nose_index, left_shoulder_index, left_elbow_index, left_wrist_index,
                      right_shoulder_index, right_elbow_index, right_wrist_index]:
            landmark = landmarks[index]
            height, width, _ = frame.shape
            cx, cy = int(landmark.x * width), int(landmark.y * height)
            cv2.circle(frame, (cx, cy), 5, (255, 0, 0), -1)  # Draw a circle at the landmark position
        
        symmetry_axis=int((int(landmarks[left_shoulder_index].x * width)+int(landmarks[right_shoulder_index].x * width))/2)
        
        # Connect wrists with elbows and elbows with shoulders
        cv2.line(frame, (int(landmarks[left_wrist_index].x * width), int(landmarks[left_wrist_index].y * height)),
                 (int(landmarks[left_elbow_index].x * width), int(landmarks[left_elbow_index].y * height)),
                 (0, 255, 0), 2)

        cv2.line(frame, (int(landmarks[right_wrist_index].x * width), int(landmarks[right_wrist_index].y * height)),
                 (int(landmarks[right_elbow_index].x * width), int(landmarks[right_elbow_index].y * height)),
                 (0, 255, 0), 2)

        cv2.line(frame, (int(landmarks[left_elbow_index].x * width), int(landmarks[left_elbow_index].y * height)),
                 (int(landmarks[left_shoulder_index].x * width), int(landmarks[left_shoulder_index].y * height)),
                 (0, 255, 0), 2)

        cv2.line(frame, (int(landmarks[right_elbow_index].x * width), int(landmarks[right_elbow_index].y * height)),
                 (int(landmarks[right_shoulder_index].x * width), int(landmarks[right_shoulder_index].y * height)),
                 (0, 255, 0), 2)
        
        cv2.line(frame, (int(landmarks[right_shoulder_index].x * width), int(landmarks[right_shoulder_index].y * height)),
                 (int(landmarks[nose_index].x * width), int(landmarks[nose_index].y * height)),
                 (0, 0, 255), 1)
            
        cv2.line(frame, (int(landmarks[left_shoulder_index].x * width), int(landmarks[left_shoulder_index].y * height)),
                 (int(landmarks[nose_index].x * width), int(landmarks[nose_index].y * height)),
                 (0, 0, 255), 1)

        cv2.line(frame, (symmetry_axis, 0),
                 (symmetry_axis, int(height)),
                 (0, 0, 255), 1)
        
        # Setup status box
        if(is_pose_symmetric(landmarks)):
            rect_color=(255,0,0)
            rect_text="Symmetric"
        else:
            rect_color=(0,0,255)
            rect_text="Non Symmetric"

        cv2.rectangle(frame, (0,0), (400,100), rect_color, -1)
        
        cv2.putText(frame,rect_text, 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)


        cv2.imshow('Mediapipe Feed', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


# Symmetry Normalized Naive

In [16]:
def is_pose_symmetric2(landmarks,height, width, symmetry_axis,threshold=300):
    left_landmarks = [landmarks[i] for i in [13,15]]  # Indices for left side
    right_landmarks = [landmarks[i] for i in [14,16]]  # Indices for right side
    
    asymmetry_score=abs(landmarks[0].x*width-symmetry_axis)
    
    for left, right in zip(left_landmarks, right_landmarks):
        asymmetry_score = asymmetry_score + abs((symmetry_axis-left.x*width) - (right.x*width - symmetry_axis))
        asymmetry_score = asymmetry_score + abs(left.y*height - right.y*height)

    arm_length=math.sqrt((landmarks[11].x-landmarks[13].x)**2+(landmarks[11].y-landmarks[13].y)**2) + math.sqrt((landmarks[12].x-landmarks[14].x)**2+(landmarks[12].y-landmarks[14].y)**2)
    
    #print(str(asymmetry_score/arm_length))
    
    if(asymmetry_score/arm_length<threshold):
        return True
    else:
        return False

In [17]:
cap = cv2.VideoCapture('GestureFeatureResources/video.mov')
#cap = cv2.VideoCapture(0)

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      
        # Make detection
        results = pose.process(image)
    
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass
        
        # Indices for the nose, shoulders, elbows, and wrists
        nose_index = 0
        left_shoulder_index = 11
        left_elbow_index = 13
        left_wrist_index = 15
        right_shoulder_index = 12
        right_elbow_index = 14
        right_wrist_index = 16
        
        # Draw circles at specific landmarks
        for index in [nose_index, left_shoulder_index, left_elbow_index, left_wrist_index,
                      right_shoulder_index, right_elbow_index, right_wrist_index]:
            landmark = landmarks[index]
            height, width, _ = frame.shape
            cx, cy = int(landmark.x * width), int(landmark.y * height)
            cv2.circle(frame, (cx, cy), 5, (255, 0, 0), -1)  # Draw a circle at the landmark position
        
        symmetry_axis=int((int(landmarks[left_shoulder_index].x * width)+int(landmarks[right_shoulder_index].x * width))/2)
        
        # Connect wrists with elbows and elbows with shoulders
        cv2.line(frame, (int(landmarks[left_wrist_index].x * width), int(landmarks[left_wrist_index].y * height)),
                 (int(landmarks[left_elbow_index].x * width), int(landmarks[left_elbow_index].y * height)),
                 (0, 255, 0), 2)

        cv2.line(frame, (int(landmarks[right_wrist_index].x * width), int(landmarks[right_wrist_index].y * height)),
                 (int(landmarks[right_elbow_index].x * width), int(landmarks[right_elbow_index].y * height)),
                 (0, 255, 0), 2)

        cv2.line(frame, (int(landmarks[left_elbow_index].x * width), int(landmarks[left_elbow_index].y * height)),
                 (int(landmarks[left_shoulder_index].x * width), int(landmarks[left_shoulder_index].y * height)),
                 (0, 255, 0), 2)

        cv2.line(frame, (int(landmarks[right_elbow_index].x * width), int(landmarks[right_elbow_index].y * height)),
                 (int(landmarks[right_shoulder_index].x * width), int(landmarks[right_shoulder_index].y * height)),
                 (0, 255, 0), 2)
        
        cv2.line(frame, (int(landmarks[right_shoulder_index].x * width), int(landmarks[right_shoulder_index].y * height)),
                 (int(landmarks[nose_index].x * width), int(landmarks[nose_index].y * height)),
                 (0, 0, 255), 1)
            
        cv2.line(frame, (int(landmarks[left_shoulder_index].x * width), int(landmarks[left_shoulder_index].y * height)),
                 (int(landmarks[nose_index].x * width), int(landmarks[nose_index].y * height)),
                 (0, 0, 255), 1)

        cv2.line(frame, (symmetry_axis, 0),
                 (symmetry_axis, int(height)),
                 (0, 0, 255), 1)
        
        # Setup status box
        if(is_pose_symmetric2(landmarks,height, width,symmetry_axis)):
            rect_color=(255,0,0)
            rect_text="Symmetric"
        else:
            rect_color=(0,0,255)
            rect_text="Non Symmetric"

        cv2.rectangle(frame, (0,0), (270,100), rect_color, -1)
        
        cv2.putText(frame,rect_text, 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)


        cv2.imshow('Mediapipe Feed', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# Symmetry Normalized Slide

In [12]:
def is_pose_symmetric3(lhand, rhand, spine_x,spine_y,width,height,threshold=0.22):
        
    h_asymmetry = abs(abs(spine_x - lhand.x*width)-abs(spine_x - rhand.x*width))/abs(abs(spine_x - lhand.x*width)+abs(spine_x - rhand.x*width))
    v_asymmetry = abs(abs(spine_y - lhand.y*height)-abs(spine_y - rhand.y*height))/abs(abs(spine_y - lhand.y*height)+abs(spine_y - rhand.y*height))

    asymmetry_score = (h_asymmetry + v_asymmetry) / 2

    if(asymmetry_score<threshold):
        return True
    else:
        return False

In [13]:
cap = cv2.VideoCapture('GestureFeatureResources/video.mov')
#cap = cv2.VideoCapture(0)

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      
        # Make detection
        results = pose.process(image)
    
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass
        
        # Indices for the nose, shoulders, elbows, and wrists
        left_wrist_index = 15
        right_wrist_index = 16
        left_hip_index = 23
        right_hip_index = 24
        
        # Draw circles at specific landmarks
        for index in [left_wrist_index, right_wrist_index, left_hip_index, right_hip_index]:
            landmark = landmarks[index]
            height, width, _ = frame.shape
            cx, cy = int(landmark.x * width), int(landmark.y * height)
            cv2.circle(frame, (cx, cy), 5, (255, 0, 0), -1)  # Draw a circle at the landmark position
        
        spine = {'x': 0.0, 'y': 0.0}
        spine["x"] = int(((landmarks[left_hip_index].x * width) + (landmarks[right_hip_index].x * width)) / 2)
        spine["y"] = int(((landmarks[left_hip_index].y * height) + (landmarks[right_hip_index].y * height)) / 2)
        cv2.circle(frame, (spine["x"], spine["y"]), 5, (0, 0, 255), -1)  # Draw a circle at the landmark position

        
        # Connect wrists with elbows and elbows with shoulders
        cv2.line(frame, (int(landmarks[left_wrist_index].x * width), int(landmarks[left_wrist_index].y * height)),
                 (int(spine["x"]), int(spine["y"])),
                 (0, 255, 0), 2)

        cv2.line(frame, (int(landmarks[right_wrist_index].x * width), int(landmarks[right_wrist_index].y * height)), 
                 (int(spine["x"]), int(spine["y"])),
                 (0, 255, 0), 2)     
        
        # Setup status box
        if(is_pose_symmetric3(landmarks[left_wrist_index],landmarks[right_wrist_index],spine["x"],spine["y"],width,height)):
            rect_color=(255,0,0)
            rect_text="Symmetric"
        else:
            rect_color=(0,0,255)
            rect_text="Non Symmetric"

        cv2.rectangle(frame, (0,0), (270,100), rect_color, -1)
        
        cv2.putText(frame,rect_text, 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)


        cv2.imshow('Mediapipe Feed', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# Compactness

Computed with the arm shape definition

In [14]:
def compute_volume_variation(lhand, rhand, spine_x,spine_y,width,height):
    volume = 0
    
    for hand in [lhand, rhand]:
        # Compute the vector from hand to base of the spine
        vector_x = spine_x - hand.x*width
        vector_y = spine_y - hand.y*height

        # Compute the magnitude of the vector
        volume = volume + math.sqrt(vector_x**2 + vector_y**2)
    
    return volume

In [15]:
cap = cv2.VideoCapture('GestureFeatureResources/video.mov')
#cap = cv2.VideoCapture(0)

previous_volumes=[]
# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      
        # Make detection
        results = pose.process(image)
    
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass
        
        # Indices for the nose, shoulders, elbows, and wrists
        left_wrist_index = 15
        right_wrist_index = 16
        left_hip_index = 23
        right_hip_index = 24
        
        # Draw circles at specific landmarks
        for index in [left_wrist_index, right_wrist_index, left_hip_index, right_hip_index]:
            landmark = landmarks[index]
            height, width, _ = frame.shape
            cx, cy = int(landmark.x * width), int(landmark.y * height)
            cv2.circle(frame, (cx, cy), 5, (255, 0, 0), -1)  # Draw a circle at the landmark position
        
        spine = {'x': 0.0, 'y': 0.0}
        spine["x"] = int(((landmarks[left_hip_index].x * width) + (landmarks[right_hip_index].x * width)) / 2)
        spine["y"] = int(((landmarks[left_hip_index].y * height) + (landmarks[right_hip_index].y * height)) / 2)
        cv2.circle(frame, (spine["x"], spine["y"]), 5, (0, 0, 255), -1)  # Draw a circle at the landmark position

        
        # Connect wrists with elbows and elbows with shoulders
        cv2.line(frame, (int(landmarks[left_wrist_index].x * width), int(landmarks[left_wrist_index].y * height)),
                 (int(spine["x"]), int(spine["y"])),
                 (0, 255, 0), 2)

        cv2.line(frame, (int(landmarks[right_wrist_index].x * width), int(landmarks[right_wrist_index].y * height)), 
                 (int(spine["x"]), int(spine["y"])),
                 (0, 255, 0), 2)        

        
        # Setup status box
        volume = compute_volume_variation(landmarks[left_wrist_index],landmarks[right_wrist_index],spine["x"],spine["y"],width,height)
        last_n_volumes = previous_volumes[-10:]
                                          
        # Count the number of elements greater than the given value
        greater_count = sum(1 for element in last_n_volumes if element > volume)

        # Count the number of elements lower than the given value
        lower_count = sum(1 for element in last_n_volumes if element < volume)

        previous_volumes.append(volume)   
                                          
        if(greater_count < lower_count):
            rect_color=(255,0,0)
            rect_text="Increasing volume"
        else:
            rect_color=(0,0,255)
            rect_text="Decreasing volume"

        cv2.rectangle(frame, (0,0), (270,100), rect_color, -1)
        
        cv2.putText(frame,rect_text, 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

        cv2.imshow('Mediapipe Feed', frame)

        previous_volume = volume
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# Appendix

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >